In [25]:
import os
import cohere
import json

co = cohere.Client(os.environ['COHERE_KEY'])

def llm(prompt):
    response = co.chat(
    	message= prompt
    )
    
    return response.text
llm('Hello world!')

'Hello! How can I help you today?'

In [50]:
def build_prompt(query,search_results):
    prompt_template = """You're a course teaching assistant, answer the QUESTION based on the CONTEXT.\nUse only the facts from the CONTEXT when asnwering the QUESTION.\nIf the CONTEXT does not contain the answer, output NONE.\n\nQUESTION: {question}\n\nCONTEXT:\n{context}"""
    context = ''
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    prompt = prompt_template.format(question=query,context=context)
    return prompt


In [59]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')
es_client.info()

ObjectApiResponse({'name': '4be90b41308e', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'dkeZzvl-RNy775CgwpCcqg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [62]:
index_settings = {
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0
        },
        "mappings": {
            "properties": {
                "text": {"type": "text"},
                "section": {"type": "text"},
                "question": {"type": "text"},
                "course": {"type": "keyword"} 
            }
        }
    }
index_name = 'course-questions'
es_client.indices.create(index=index_name,body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [63]:
from tqdm.auto import tqdm
def index_course_document(index_name,json_document_path):
    documents = []
    with open(json_document_path, 'rt') as f_in:
        docs_raw = json.load(f_in)
    
    for course_dict in docs_raw:
        for doc in course_dict['documents']:
            doc['course'] = course_dict['course']
            documents.append(doc)
    
    for doc in tqdm(documents):
        es_client.index(index=index_name,document=doc)

index_course_document(index_name,'documents.json')

  0%|          | 0/948 [00:00<?, ?it/s]

In [64]:
documents[2]

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

In [65]:
query ='I just discovered the course. Can I still join it?'
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}
response = es_client.search(index=index_name,body=search_query)
response

ObjectApiResponse({'took': 197, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 405, 'relation': 'eq'}, 'max_score': 72.849266, 'hits': [{'_index': 'course-questions', '_id': 'Lv0fcJABRyfhs0jvpfp8', '_score': 72.849266, '_source': {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'section': 'General course-related questions', 'question': 'Course - Can I still join the course after the start date?', 'course': 'data-engineering-zoomcamp'}}, {'_index': 'course-questions', '_id': 'M_0fcJABRyfhs0jvpvoU', '_score': 54.057133, '_source': {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next

In [66]:
print(response.meta)

ApiResponseMeta(status=200, http_version='1.1', headers={'X-elastic-product': 'Elasticsearch', 'content-type': 'application/vnd.elasticsearch+json;compatible-with=8', 'content-length': '2847'}, duration=0.200730562210083, node=NodeConfig(scheme='http', host='localhost', port=9200, path_prefix='', headers={'user-agent': 'elasticsearch-py/8.14.0 (Python/3.10.13; elastic-transport/8.13.1)'}, connections_per_node=10, request_timeout=10.0, http_compress=False, verify_certs=True, ca_certs=None, client_cert=None, client_key=None, ssl_assert_hostname=None, ssl_assert_fingerprint=None, ssl_version=None, ssl_context=None, ssl_show_warn=True, _extras={}))


In [17]:
def elastic_search(query):
    result_docs = []
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name,body=search_query)
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [67]:
query ='I just discovered the course. Can I still join it?'
search_results = elastic_search(query)
prompt = build_prompt(query, search_results)
len(prompt)

2275

In [68]:
len(prompt.split(' '))

339

In [26]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [42]:
query ='I just discovered the course. Can I still join it?'
print(rag(query))

Yes, you can still join the course even after it has started. You are eligible to submit homework assignments, but be mindful of the deadlines for turning in final projects.
